In [1]:
import os
from pathlib import Path
import yfinance as yf
import json
import matplotlib.pyplot as plt
from typing import List

import pandas as pd
import numpy as np

In [2]:
tickers = {
    "tech": ["AAPL", "MSFT", "TSLA"],
    "healthcare": ["JNJ", "PFE", "MRK"],
    "financials": ["JPM", "BAC", "WFC"],
    "energy": ["XOM", "CVX", "SLB"],
    "consumer": ["PG", "KO", "WMT"],
    "commodities": ["GC=F", "CL=F"]
}

In [ ]:
for sector, symbols in tickers.items():
    print(f"Fetching data for {sector} sector...")
    for symbol in symbols:
        try:
            ticker_data = yf.Ticker(symbol)
            hist = ticker_data.history(start="2010-01-01")
            hist.to_csv(f"historical_data/{symbol}_data.csv")
            print(f"{symbol} data fetched successfully.")
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    print(f"Finished fetching data for {sector} sector.\n")

In [ ]:
#!/usr/bin/env python3
"""
split_csv_by_date.py

For each CSV in `file_paths`, read it with pandas, split into train/test based on the 'Date' column,
and write out two new files:
    [original_filename]_train.csv   (all rows with Date < 2024-01-01)
    [original_filename]_test.csv    (all rows with Date >= 2024-01-01)

Usage:
    python split_csv_by_date.py

Adjust `file_paths` below (or modify this script to accept command-line arguments).
"""
# ----------------------------------------------------------------------
# STEP 1: Specify your CSV files here. You can list absolute or relative paths.
# ----------------------------------------------------------------------
file_paths = Path("historical_data").glob("*.csv")

# ----------------------------------------------------------------------
# STEP 2: Define the cutoff date for the test set
# ----------------------------------------------------------------------
# We want test = all rows with Date >= January 1, 2024
TEST_CUTOFF = pd.Timestamp("2024-01-01", tz="UTC")  # Ensure timezone consistency if needed

# ----------------------------------------------------------------------
# STEP 3: Loop over each CSV, read, split, and write out train/test
# ----------------------------------------------------------------------
for csv_path in file_paths:

    # 2. Read the CSV, parsing 'Date' as datetime:
    try:
        df = pd.read_csv(csv_path, parse_dates=["Date"])
    except Exception as e:
        print(f"Error reading '{csv_path}': {e}")
        continue

    # 3. Verify that the 'Date' column exists and is datetime64:
    if "Date" not in df.columns:
        print(f"Warning: 'Date' column not found in '{csv_path}'. Skipping.")
        continue

    if not pd.api.types.is_datetime64_any_dtype(df["Date"]):
        # If pandas did not parse it as datetime, try to convert explicitly:
        try:
            df["Date"] = pd.to_datetime(df["Date"], errors="raise", utc=True)
        except Exception as e:
            print(f"Could not convert 'Date' to datetime in '{csv_path}': {e}")
            continue

    # 4. Split into train/test
    mask_test = df["Date"] >= TEST_CUTOFF
    df_test = df.loc[mask_test].copy()
    df_train = df.loc[~mask_test].copy()

    # 5. Construct output filenames:
    base, ext = os.path.splitext(csv_path)
    train_path = f"{base}_train{ext}"
    test_path  = f"{base}_test{ext}"

    # 6. Write them out (index=False to avoid writing the index column unless you want it):
    try:
        df_train.to_csv(train_path, index=False)
        df_test.to_csv(test_path, index=False)
        print(f"Split '{csv_path}' →")
        print(f"    TRAIN ({len(df_train)} rows) → {train_path}")
        print(f"    TEST  ({len(df_test)} rows) → {test_path}")
    except Exception as e:
        print(f"Error writing split files for '{csv_path}': {e}")

In [ ]:
def moving_average_crossover(df: pd.DataFrame,
                             short_window: int = 20,
                             long_window: int = 50) -> pd.DataFrame:
    """
    Compute a simple moving‐average crossover strategy on `df`.
    Assumes `df` has a 'Date' column (or datetime index) and a 'Close' column.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain 'Date' (or be indexed by datetime) and 'Close'.
    short_window : int
        Window size for the short SMA (e.g., 20 days).
    long_window : int
        Window size for the long SMA (e.g., 50 days).

    Returns
    -------
    pd.DataFrame
        Original DataFrame with the following added columns:
          - 'SMA_short' : short‐period moving average
          - 'SMA_long'  : long‐period moving average
          - 'Signal'    : 1 when short crosses above long,
                          -1 when short crosses below long, else 0
    """

    df = df.copy()

    # Ensure 'Date' is a datetime type and set as index if not already
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors="raise", utc=True)
        df.set_index("Date", inplace=True)
    elif not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame must have a 'Date' column or be indexed by datetime.")

    # 1) Compute Simple Moving Averages
    df["SMA_short"] = df["Close"].rolling(window=short_window, min_periods=1).mean()
    df["SMA_long"]  = df["Close"].rolling(window=long_window, min_periods=1).mean()

    # 2) Generate raw crossover signals: 
    #    - When SMA_short > SMA_long → 1 (long), when SMA_short < SMA_long → 0
    df["Crossover"] = np.where(df["SMA_short"] > df["SMA_long"], 1, 0)

    # 3) The actual “Signal” is the change in Crossover:
    #    - A rising edge (0→1) means a buy signal (1)
    #    - A falling edge (1→0) means a sell signal (−1)
    df["Signal"] = df["Crossover"].diff().fillna(0)
    df["Signal"] = df["Signal"].map({1.0: 1, -1.0: -1}).fillna(0).astype(int)

    # Rows where no crossover event happen remain at 0
    # Cleanup temporary column if you don’t need it:
    df.drop(columns=["Crossover"], inplace=True)

    return df

In [ ]:
def bollinger_bands_strategy(df: pd.DataFrame,
                             window: int = 20,
                             num_std: float = 2.0) -> pd.DataFrame:
    """
    Compute a simple Bollinger Bands breakout strategy on `df`.
    Assumes `df` has a 'Date' column (or datetime index) and a 'Close' column.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain 'Date' (or be indexed by datetime) and 'Close'.
    window : int
        Rolling window size for moving average and standard deviation.
    num_std : float
        Number of standard deviations for upper/lower bands.

    Returns
    -------
    pd.DataFrame
        Original DataFrame with these added columns:
          - 'BB_mid'   : rolling mean (middle band)
          - 'BB_std'   : rolling standard deviation
          - 'BB_upper' : BB_mid + num_std * BB_std
          - 'BB_lower' : BB_mid − num_std * BB_std
          - 'Signal'   :  1 when Close < BB_lower (buy)
                          -1 when Close > BB_upper (sell), else 0
    """

    df = df.copy()

    # Ensure 'Date' is datetime and set as index if needed
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors="raise", utc=True)
        df.set_index("Date", inplace=True)
    elif not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame must have a 'Date' column or be indexed by datetime.")

    # 1) Compute rolling mean (mid) and rolling std (std) over 'window'
    df["BB_mid"] = df["Close"].rolling(window=window, min_periods=1).mean()
    df["BB_std"] = df["Close"].rolling(window=window, min_periods=1).std(ddof=0)

    # 2) Compute upper and lower bands
    df["BB_upper"] = df["BB_mid"] + (num_std * df["BB_std"])
    df["BB_lower"] = df["BB_mid"] - (num_std * df["BB_std"])

    # 3) Generate signals:
    #    - Buy (1)  when price closes below the lower band
    #    - Sell (-1) when price closes above the upper band
    conditions = [
        df["Close"] < df["BB_lower"],  # price has broken below lower band
        df["Close"] > df["BB_upper"],  # price has broken above upper band
    ]
    choices = [1, -1]

    df["Signal"] = np.select(conditions, choices, default=0).astype(int)

    return df

In [ ]:
def backtest_with_shorts(
    df: pd.DataFrame,
    price_col: str = "Close",
    signal_col: str = "Signal"
) -> List[float]:
    """
    Backtest a strategy that can go long (signal=+1), short (signal=-1), or flat (signal=0).
    For each change in signal, it closes any open position and (if new signal != 0) opens a new position.
    Returns a list of percent PnLs for each completed trade.

    Parameters
    ----------
    df : pd.DataFrame
        Must be sorted in ascending date order and contain:
          - price_col   : price at which to execute trades (float)
          - signal_col  : integer signal (+1=open long, -1=open short, 0=flat)
    price_col : str
        Name of the column with execution prices.
    signal_col : str
        Name of the column with signals (+1, -1, 0).

    Returns
    -------
    List[float]
        Percent PnLs for each round-trip (long or short) trade.
    """
    pnls: List[float] = []
    current_position = 0   # +1 for long, -1 for short, 0 for flat
    entry_price = None     # price at which the current position was opened

    for idx, row in df.iterrows():
        sig = int(row[signal_col])
        price = float(row[price_col])

        # If the signal changes from whatever we currently hold:
        if sig != current_position:
            # 1) Close any existing position
            if current_position != 0 and entry_price is not None:
                if current_position == 1:
                    # closing a long
                    pnl = (price - entry_price) / entry_price * 100.0
                else:  # current_position == -1, closing a short
                    pnl = (entry_price - price) / entry_price * 100.0

                pnls.append(pnl)
                entry_price = None
                current_position = 0

            # 2) Open a new position if sig != 0
            if sig != 0:
                current_position = sig
                entry_price = price

        # If sig == current_position, do nothing (hold the position)
        # If sig == 0 and current_position == 0, do nothing (remain flat)

    # At the end: if a position is still open, close it at the last price
    if current_position != 0 and entry_price is not None:
        last_price = float(df.iloc[-1][price_col])
        if current_position == 1:
            pnl = (last_price - entry_price) / entry_price * 100.0
        else:  # current_position == -1
            pnl = (entry_price - last_price) / entry_price * 100.0

        pnls.append(pnl)

    return pnls

In [ ]:
for sector, symbols in tickers.items():
    print(f"Processing strategies for {sector} sector...")
    for symbol in symbols:
        try:
            file_path = Path(f"historical_data/{symbol}_data.csv")
            if not file_path.exists():
                print(f"File {file_path} does not exist. Skipping.")
                continue

            df = pd.read_csv(file_path)

            # Apply moving average crossover strategy
            df_ma = moving_average_crossover(df)
            ma_pnls = backtest_with_shorts(df_ma)
            print(f"{symbol} MA Strategy PnLs: {ma_pnls}")

            with open(f"backtest_pnl/{symbol}_ma_strategy.json", "w") as f:
                json.dump(ma_pnls, f)

            # Apply Bollinger Bands strategy
            df_bb = bollinger_bands_strategy(df)
            bb_pnls = backtest_with_shorts(df_bb)
            print(f"{symbol} BB Strategy PnLs: {bb_pnls}")

            with open(f"backtest_pnl/{symbol}_bb_strategy.json", "w") as f:
                json.dump(bb_pnls, f)

        except Exception as e:
            print(f"Error processing {symbol}: {e}")
    print(f"Finished processing strategies for {sector} sector.\n")

In [ ]:
# # Cumulative PnL plot
# cumulative_pnl = np.cumsum(backtest_results)
# plt.figure(figsize=(12, 6))
# plt.plot(cumulative_pnl, marker='o', linestyle='-', color='blue')
# plt.title("Cumulative PnL from Backtest")
# plt.xlabel("Trade Number")
# plt.ylabel("Cumulative PnL (%)")
# plt.grid()
# plt.axhline(0, color='red', linestyle='--', linewidth=1)
# plt.show()

In [3]:
pnls_df = pd.DataFrame()
for sector, symbols in tickers.items():
    print(f"Processing strategies for {sector} sector...")
    for symbol in symbols:
        try:
            backtest_ema_path = Path(f"backtest_pnl/{symbol}_ma_strategy.json")
            if not backtest_ema_path.exists():
                print(f"File {backtest_ema_path} does not exist. Skipping.")
                continue
            with open(backtest_ema_path, "r") as f:
                ma_pnls = json.load(f)
            
            backtest_bb_path = Path(f"backtest_pnl/{symbol}_bb_strategy.json")
            if not backtest_bb_path.exists():
                print(f"File {backtest_bb_path} does not exist. Skipping.")
                continue
            with open(backtest_bb_path, "r") as f:
                bb_pnls = json.load(f)
            
            # Handle different lengths by padding with NaN
            max_len = max(len(ma_pnls), len(bb_pnls))
            ma_pnls_padded = ma_pnls + [np.nan] * (max_len - len(ma_pnls))
            bb_pnls_padded = bb_pnls + [np.nan] * (max_len - len(bb_pnls))
            
            # Create a DataFrame for the PnLs
            symbol_df = pd.DataFrame({
                f"{symbol}_MA_PnL": ma_pnls_padded,
                f"{symbol}_BB_PnL": bb_pnls_padded
            })
            pnls_df = pd.concat([pnls_df, symbol_df], axis=1)
        except Exception as e:
            print(f"Error processing {symbol}: {e}")

Processing strategies for tech sector...
Processing strategies for healthcare sector...
Processing strategies for financials sector...
Processing strategies for energy sector...
Processing strategies for consumer sector...
Processing strategies for commodities sector...


In [4]:
pnls_df.head()

,AAPL_MA_PnL,AAPL_BB_PnL,MSFT_MA_PnL,MSFT_BB_PnL,TSLA_MA_PnL,TSLA_BB_PnL,JNJ_MA_PnL,JNJ_BB_PnL,PFE_MA_PnL,PFE_BB_PnL,...,PG_MA_PnL,PG_BB_PnL,KO_MA_PnL,KO_BB_PnL,WMT_MA_PnL,WMT_BB_PnL,GC=F_MA_PnL,GC=F_BB_PnL,CL=F_MA_PnL,CL=F_BB_PnL
0,0.816061,2.690274,-0.769750,1.243074,-4.205355,-3.730999,-0.108878,0.031633,-2.308519,-0.884978,...,-0.397425,0.210761,0.182572,-0.036779,-0.236267,0.144380,-1.265593,-0.680149,-0.464156,0.965926
1,-0.303668,1.390171,4.001428,0.816195,-5.392702,0.047514,1.082437,1.035184,0.449443,0.299982,...,0.062898,0.951788,-0.707468,0.339811,-0.186418,-1.145068,-0.829318,0.310611,0.942780,-1.391356
2,-1.422994,0.066989,-1.643776,0.067510,2.726774,4.731550,0.657115,0.359115,1.732693,0.865548,...,-0.602409,0.437496,0.903051,1.229680,0.604451,-0.836271,0.048565,0.376628,2.903579,-2.531035
3,-0.355216,-2.991509,-0.461797,-1.087689,-0.743629,7.142813,-0.033446,-0.076864,0.178241,0.348231,...,-0.083362,1.425687,-0.764719,0.926798,-0.222391,0.136422,-0.052537,-0.807075,3.721551,-3.963984
4,0.190662,-0.440293,0.409285,0.095695,-3.927946,1.968886,0.393070,0.562615,0.056850,1.867466,...,-0.610366,0.368017,0.015341,0.707468,-0.199503,0.835350,0.454323,0.974494,0.495054,-3.072192


In [32]:
cov_matrix = pnls_df.cov()

In [33]:
cov_matrix

,AAPL_MA_PnL,AAPL_BB_PnL,MSFT_MA_PnL,MSFT_BB_PnL,TSLA_MA_PnL,TSLA_BB_PnL,JNJ_MA_PnL,JNJ_BB_PnL,PFE_MA_PnL,PFE_BB_PnL,...,PG_MA_PnL,PG_BB_PnL,KO_MA_PnL,KO_BB_PnL,WMT_MA_PnL,WMT_BB_PnL,GC=F_MA_PnL,GC=F_BB_PnL,CL=F_MA_PnL,CL=F_BB_PnL
AAPL_MA_PnL,3.667556,-0.125199,0.542154,-0.606126,-0.717352,-0.623457,-0.050975,0.242392,-0.312502,0.345672,...,0.055325,0.145325,0.125994,-0.195539,-0.066994,-0.244273,-0.078336,-0.286063,0.953482,0.557050
AAPL_BB_PnL,-0.125199,4.447259,1.302169,0.755701,0.519697,0.190628,0.106580,0.176511,-0.189890,-0.725707,...,-0.050691,-0.082639,0.114691,-0.063432,0.242349,-0.392476,0.274900,0.040098,-0.818917,-0.366846
MSFT_MA_PnL,0.542154,1.302169,6.473246,0.460565,-0.370707,-1.062784,0.110909,-0.317865,-0.079939,0.563283,...,-1.047560,0.181377,0.349260,0.611516,-0.515899,0.058866,0.047975,0.002195,-0.145529,0.000619
MSFT_BB_PnL,-0.606126,0.755701,0.460565,5.721915,0.496046,1.129577,-0.318298,0.138730,0.221254,-0.485812,...,-0.009737,-0.243160,0.054908,-0.131851,0.259603,-0.318064,0.091572,0.029404,0.000302,0.822581
TSLA_MA_PnL,-0.717352,0.519697,-0.370707,0.496046,16.488339,0.022888,-0.458806,-0.309549,0.397107,-0.140531,...,0.023575,0.494073,1.087990,0.324065,1.378099,0.425829,0.317234,0.015115,-1.236869,4.100163
TSLA_BB_PnL,-0.623457,0.190628,-1.062784,1.129577,0.022888,41.761053,0.223250,-0.212756,0.267484,-0.782780,...,0.383530,-0.614147,0.021096,0.273962,0.453131,0.168035,-0.435256,0.405899,2.612401,7.126190
JNJ_MA_PnL,-0.050975,0.106580,0.110909,-0.318298,-0.458806,0.223250,0.915408,0.168527,-0.069256,-0.050180,...,-0.066500,-0.042949,0.042389,-0.033956,0.068464,0.161098,-0.141237,0.186294,0.094850,-0.259344
JNJ_BB_PnL,0.242392,0.176511,-0.317865,0.138730,-0.309549,-0.212756,0.168527,2.076763,0.030083,0.152943,...,0.105355,0.028410,-0.195383,-0.320455,0.239612,-0.254333,-0.040279,-0.051400,0.289524,0.350079
PFE_MA_PnL,-0.312502,-0.189890,-0.079939,0.221254,0.397107,0.267484,-0.069256,0.030083,1.362529,-0.080156,...,0.156042,-0.048556,-0.108878,0.088093,0.518957,0.044925,-0.014711,0.317789,0.088230,0.106990
PFE_BB_PnL,0.345672,-0.725707,0.563283,-0.485812,-0.140531,-0.782780,-0.050180,0.152943,-0.080156,3.569646,...,-0.377744,-0.003870,0.162124,0.057369,-0.098818,-0.171256,0.085277,-0.011635,0.198112,-0.456297


In [7]:
# Save the covariance matrix to a CSV file for later use
cov_matrix.to_csv("covariance_matrix.csv")

In [ ]:
# Save vectorized PnLs mean for each strategy
mean_pnls = pnls_df.mean().to_dict()
with open("mean_pnls.json", "w") as f:
    json.dump(mean_pnls, f)



In [ ]:
mean_pnls = np.array(list(mean_pnls.values()))
cov_matrix = cov_matrix.values

In [16]:
w = np.array([1] * cov_matrix.shape[0])

In [ ]:
np.block([[w], [mean_pnls.T]])

(2, 34)

In [31]:
np.ones((1, cov_matrix.shape[0])) @ w == np.array([1.0])


array([False])

In [27]:
w = np.random.rand(cov_matrix.shape[0])

(34,)

In [29]:
np.array([1] * cov_matrix.shape[0]).shape

(34,)

In [35]:
# Load the COV matrix from a CSV file
cov_matrix_path = 'strategy_portfolio_opt/covariance_matrix.csv'
cov_mat_df = pd.read_csv(cov_matrix_path, index_col=0)
cov_mat = cov_mat_df.values

# Load the expected returns from a CSV file
expected_returns_path = 'strategy_portfolio_opt/mean_pnls.json'
expected_returns_dict = pd.read_json(expected_returns_path, orient='index')
expected_returns = expected_returns_dict.values.flatten()

# Asset the indexes for the covariance matrix and expected returns
asset_indexes = cov_mat_df.index.to_list()
if not np.array_equal(asset_indexes, expected_returns_dict.index.to_list()):
    raise ValueError("Asset indexes in covariance matrix and expected returns do not match.")